<a href="https://colab.research.google.com/github/arohanajit/candidate-screening-system/blob/master/candidate-screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
import os
import sklearn
from sklearn import preprocessing
from google.colab import files

In [0]:
role = input("Enter Role for potential Candidate(Web/ML): ")
while(role!='Web' and role!='ML'):
    print("Invalid Input: Try Again")
    role = input("Enter Role for potential Candidate(Web/ML): ")
print("Sucessful!")
vacancy = int(input("Enter no. of vacancies: "))

In [0]:
if 'mavoix.xlsx' not in os.listdir():
    #uploaded = files.upload()
    #for fn in uploaded.keys():
        # print('User uploaded file "{name}" with length {length} bytes'.format(
        # name=fn, length=len(uploaded[fn])))
    os.system('wget https://raw.githubusercontent.com/arohanajit/candidate-screening-system/master/mavoix.xlsx')
    print('dataset downloaded.')
else:
    print('dataset present')

In [0]:
df = pd.read_excel('mavoix.xlsx')
ID = list(df['Application_ID'])
df = df.drop(['Application_ID'],axis=1)
df = df.drop(['Unnamed: 10'],axis=1)
df = df.fillna('0')
df = df.replace('Banglore','Bangalore')
df.head()

In [0]:
for i in list(df.columns):
    if '(out of 3)' in i:
        j=i.strip('(out of 3)')
        df.rename(columns = {i:j}, inplace=True)

In [0]:
def clean(x):
    x = x.split(',')
    return len(x)

df['Other skills'] = df['Other skills'].apply(clean)

In [0]:
df['Stream'] = df['Stream'].str.lower()
def clean(x):
    if 'computer' in x:
        if len(x.split(" ")) > 2:
            x = x.replace(x,'computer science')
    if 'mca' in x:
        x = x.replace(x,'computer application')
    if 'it' in x:
        x = x.replace(x,'information technology')
    if 'cs' in x:
        x = x.replace(x,'computer science')
    return x
df['Stream'] = df['Stream'].apply(clean)

In [0]:
def clean(x):
    if '/' in x:
        total = float(x[x.index('/')+1:])
        score = float(x[:x.index('/')])
        if total==10:
            score*=9.5
        return score
    else:
        return 0
df['Performance_UG'] = df['Performance_UG'].apply(clean)
df['Performance_PG'] = df['Performance_PG'].apply(clean)
df['Performance_12'] = df['Performance_12'].apply(clean)
df['Performance_10'] = df['Performance_10'].apply(clean)

In [0]:
ccity = df['Current City'].unique()
grad_year = df['Current Year Of Graduation'].unique()
degree = df['Degree'].unique()
stream = df['Stream'].unique()

In [0]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1)) 
label_col = ['Current City','Degree','Stream','Current Year Of Graduation']
col_to_be_scaled = list(set(list(df.columns)) - set(label_col))
print(col_to_be_scaled)
df[col_to_be_scaled] = min_max_scaler.fit_transform(df[col_to_be_scaled])
df.head()

In [0]:
#For DataScience
if role=='ML':
    pref_cities = {'Bangalore':4,'Hyderabad':2,'Visakhapatnam':2}
    pref_cities = dict(sorted(pref_cities.items(), key=lambda x: x[1], reverse=True))
    pref_degree = {'Science':1,'Technology':1,'Computer':2,'Data':3,'Analytics':3,'Tech':1}
    pref_degree = dict(sorted(pref_degree.items(), key=lambda x: x[1], reverse=True))
    pref_stream = {'computer':2,'data':3,'artificial':4,'machine':4,'deep':4,'information':2,'electronics':1}
    pref_stream = dict(sorted(pref_stream.items(), key=lambda x: x[1], reverse=True))
    pref_skills = {'Python':8,'R Programming':6,'Deep Learning':10,'PHP':3,'MySQL':5,'HTML':3,'CSS':3,'JavaScrip':3,'AJAX':3,
                'Bootstrap':3,'MongoDB':5,'Node.js':3,'ReactJS':3}
    pref_skills = dict(sorted(pref_skills.items(), key=lambda x: x[1], reverse=True))
    pref_gradyr = {2017: 2, 2019: 3, 2018: 3, 2020: 5, 2021: 5, 2016: 2, 2015: 2, 2022: 4, 2012: 2, 2013: 2, 2008: 1, 2007: 1, 2014: 1, 2006: 1, 2004: 1, 2023: 3}
    pref_gradyr = dict(sorted(pref_gradyr.items(), key=lambda x: x[1], reverse=True))
elif role=='Web':
    pref_cities = {'Bangalore':4,'Hyderabad':2,'Visakhapatnam':2}
    pref_cities = dict(sorted(pref_cities.items(), key=lambda x: x[1], reverse=True))
    pref_degree = {'Science':3,'Technology':3,'Computer':3,'Data':2,'Analytics':2,'Tech':3}
    pref_degree = dict(sorted(pref_degree.items(), key=lambda x: x[1], reverse=True))
    pref_stream = {'computer':4,'data':3,'artificial':3,'machine':3,'deep':3,'information':4,'electronics':4}
    pref_stream = dict(sorted(pref_stream.items(), key=lambda x: x[1], reverse=True))
    pref_skills = {'Python':7,'R Programming':3,'Deep Learning':3,'PHP':7,'MySQL':7,'HTML':5,'CSS':6,'JavaScrip':8,'AJAX':8,
                'Bootstrap':6,'MongoDB':8,'Node.js':10,'ReactJS':10}
    pref_skills = dict(sorted(pref_skills.items(), key=lambda x: x[1], reverse=True))
    pref_gradyr = {2017: 2, 2019: 3, 2018: 3, 2020: 5, 2021: 5, 2016: 2, 2015: 2, 2022: 4, 2012: 2, 2013: 2, 2008: 1, 2007: 1, 2014: 1, 2006: 1, 2004: 1, 2023: 3}
    pref_gradyr = dict(sorted(pref_gradyr.items(), key=lambda x: x[1], reverse=True))

In [0]:
cols = list(df.columns)
print(cols)
if 'Bangalore' in list(pref_cities.keys()):
    print('yes')

In [0]:
def giveScore(n,m,x):
    temp1 = n&m
    val = []
    for i in temp1:
        if x==1:
            val.append(pref_degree[i])
        else:
            val.append(pref_stream[i])
    if val:
        return max(val)
    else:
        return 0

In [0]:
a = df.iloc[1]
print(a['Current Year Of Graduation'])
print(pref_gradyr[a['Current Year Of Graduation']])

In [0]:
total_score = []
for _,j in df.iterrows():
    Score = 0
    if j['Current City'] in list(pref_cities.keys()):
        Score+=pref_cities[j['Current City']]
    for i in list(pref_skills.keys()):
        Score+=j[i]*pref_skills[i]
    Score+=0.3*j['Other skills']
    temp1 = (set(j['Degree'].split(" "))&set(pref_degree.keys()))
    Score+=giveScore(set(j['Degree'].split(" ")),set(pref_degree.keys()),1)
    Score+=giveScore(set(j['Stream'].split(" ")),set(pref_stream.keys()),2)
    Score+=j['Performance_PG']*5
    Score+=j['Performance_UG']*5
    Score+=j['Performance_12']*4
    Score+=j['Performance_10']*3
    Score+=pref_gradyr[j['Current Year Of Graduation']]

    total_score.append(Score)

df["Candidate Score"] = total_score
    



In [0]:
df.insert(0,column="Application_ID",value=ID)
df = df.sort_values(by=['Candidate Score'],ascending=False)
df.head()

In [0]:
print("Top {} recommened candidates for {} are: {}".format(vacancy,role,list(df.iloc[:vacancy]['Application_ID'])))